<a href="https://colab.research.google.com/github/KJFUENTES/DS-Capstone-Coursera/blob/master/Interactive%20Dashboard%20with%20PlotyDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-09-28 08:23:38--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-09-28 08:23:38 (322 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [ ]:
!pip install jupyter-dash

     |████████████████████████████████| 7.3 MB 8.7 MB/s 
     |████████████████████████████████| 23.9 MB 1.2 MB/s 
     |████████████████████████████████| 357 kB 70.3 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=804f7972451a952a485aa17b351211a4ffa8b797215344b049a9e1af0a9b537e
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=a95462fce72e64c8782b48a7b36aa641fefb7f581e32f8baeb58c442cf5f2ac0
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=e0c458e088dec44ce910deb4d423d4dc09b08568305b1383dc8217ce2209f59f
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """


In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                html.Div([
                                          html.Div(
                                              [
                                               html.H2('Launch Site:', style={'margin-right': '2em'}),
                                               ]
                                               ),
                                          dcc.Dropdown(id='site-dropdown',
                                                       options=[{'label': 'All Sites', 'value': 'ALL'},
                                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                                       value='ALL',
                                                       placeholder='place holder here',
                                                       searchable=True)
                                          ]),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
  filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
  if entered_site == 'ALL':
    fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')
    return fig
  else:
    fig = px.pie(filtered_df, names='class', title='Total Success Launches for site '+str(entered_site))
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value")])

def get_scatter_chart(entered_site, slider_range):
  low, high = slider_range
  filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
  if entered_site == 'ALL':
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    fig = px.scatter(spacex_df[mask], x="Payload Mass (kg)", y="class", color='Booster Version Category', title='Correlation between Payload and Success for all sites')
    return fig
  else:
    mask = (filtered_df['Payload Mass (kg)'] > low) & (filtered_df['Payload Mass (kg)'] < high)
    fig = px.scatter(filtered_df[mask], x="Payload Mass (kg)", y="class", color='Booster Version Category', title='Correlation between Payload and Success for '+str(entered_site))
    return fig

# Run the app
if __name__ == '__main__':
  app.run_server(mode="external")

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
"""
Which site has the largest successful launches?
KSC LC-39A
Which site has the highest launch success rate?
KSC LC-39A
Which payload range(s) has the highest launch success rate?
2K-4K
Which payload range(s) has the lowest launch success rate?
6K-8K
Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?
FT
"""

'\nWhich site has the largest successful launches?\nKSC LC-39A\nWhich site has the highest launch success rate?\nCCAFS SLC-40\nWhich payload range(s) has the highest launch success rate?\n2K-4K\nWhich payload range(s) has the lowest launch success rate?\n6K-8K\nWhich F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest\nFT\n'